In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime

from dateutil.parser import parse
from datetime import timedelta
%matplotlib inline

# set random
np.random.seed(42)

# seaborn style
sns.set(style="darkgrid")
sns.set_palette("hls", 8)

# read in data
loans = pd.read_csv("kiva_loans.csv")

#mpi_locations = pd.read_csv("kiva_mpi_region_locations.csv")
#loan_themes = pd.read_csv("loan_theme_ids.csv")
#themes_by_region = pd.read_csv("loan_themes_by_region.csv")

In [113]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 20 columns):
id                    671205 non-null int64
funded_amount         671205 non-null float64
loan_amount           671205 non-null float64
activity              671205 non-null object
sector                671205 non-null object
use                   666977 non-null object
country_code          671197 non-null object
country               671205 non-null object
region                614405 non-null object
currency              671205 non-null object
partner_id            657698 non-null float64
posted_time           671205 non-null object
disbursed_time        668809 non-null object
funded_time           622874 non-null object
term_in_months        671205 non-null float64
lender_count          671205 non-null int64
tags                  499789 non-null object
borrower_genders      666984 non-null object
repayment_interval    671205 non-null object
date                  671205 non

671,205 entries. Pretty big dataset.

In [114]:
# drop nan values
loans.dropna(subset=["posted_time", "funded_time"], how='any', axis=0, inplace=True)

# set to datetime
loans["posted_time"] = pd.to_datetime(loans["posted_time"], infer_datetime_format=True)
loans["funded_time"] = pd.to_datetime(loans["funded_time"], infer_datetime_format=True)

### Caclulate total number of hours it took to fund loan

In [115]:
loans_subset = loans[0:15]

# create column of how many hours a loan took to fund
hours_to_fund = []
for index, row in loans_subset.iterrows():
    difference = row.funded_time - row.posted_time
    print(difference)
    diff_in_hours = round((difference.seconds * 60 * 60) + (difference.days / 24))
    hours_to_fund.append(diff_in_hours)
    
loans_subset["hours_to_fund"] = hours_to_fund


# TAKES SUPER LONG WITH WHOLE DATASET:
# hours_to_fund = []
# for index, row in loans.iterrows():
#     difference = row.funded_time - row.posted_time
#     print(difference)
#     diff_in_hours = round((difference.seconds * 60 * 60) + (difference.days / 24))
#     hours_to_fund.append(diff_in_hours)
    
# loans["hours_to_fund"] = hours_to_fund

1 days 03:53:53
1 days 02:26:15
0 days 06:03:29
0 days 04:56:49
0 days 07:25:32
28 days 04:08:38
0 days 07:27:07
9 days 06:32:43
0 days 06:57:38
2 days 10:04:01
0 days 04:46:16
1 days 21:15:43
6 days 12:30:17
1 days 07:49:39
0 days 07:33:38


/Users/mariah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### Calculate day/month/weekday each loan was posted

In [118]:
loan_timing = pd.DataFrame(loans_subset["id"])

loan_timing['hour_posted'] = loans_subset.apply(lambda row: row.posted_time.hour, axis=1)
loan_timing['weekday_posted'] = loans_subset.apply(lambda row: row.posted_time.weekday(), axis=1)
loan_timing['month_posted'] = loans_subset.apply(lambda row: row.posted_time.month, axis=1)

loans_with_timing = pd.merge(loan_timing, loans_subset, on="id")